In [1]:
import numpy as np
import matplotlib.pyplot as plt
X_train = np.load("reshaped_14.npy")

In [2]:
mean = 2
cov  = 2
samples = 60000
# input_size = 784
input_size = 196
# input_size = 100
Xf = X_train.reshape(samples,input_size)
Xf = Xf/255.0
# defining functions

np.random.seed(420)

def sigm(x):
    return 1/(1+np.exp(-x))

def diff_sigm(x):
    return (sigm(x)-(sigm(x)**2))

def relu(x):
    x[x<0]=0
    return x

def relu(x, derivative=False):
    res = x
    return res * (res > 0)

def diff_relu(x):
    res = x
    return res * (res > 1)

def diff_SSE(y,x,batch_size):
    return (2*(y-x))/batch_size

def diff_CCE(y,x):
    epsilon = 10e-8
    loss = -y * np.log(x + epsilon) - (1 - y) * np.log(1 - x + epsilon)
    return loss

def layer(x,W,b):
    return np.matmul(x,W)+b

def backprop_layer_relu(prev_delta,out_prev,weights_prev):
    sn = -(diff_relu(out_prev)*np.matmul(prev_delta,weights_prev.T))
    return sn

def backprop_layer_sigm(prev_delta,out_prev,weights_prev):
    sn = -(diff_sigm(out_prev)*np.matmul(prev_delta,weights_prev.T))
    return sn

Wc = np.random.normal(0,1e-3,(input_size,mean))
Bc = np.random.normal(0,1e-3,(1,mean))

Wm = np.random.normal(0,1e-3,(input_size,mean))
Bm = np.random.normal(0,1e-3,(1,mean))

Wd = np.random.normal(0,1e-3,(mean,input_size))
Bd = np.random.normal(0,1e-3,(1,input_size))

In [ ]:
#Here the covariance is log coavriance but named as coavriance for peace of mind while coding
lr = 1e-3
epochs = 300
batch_size = 100
Lambda = 0.5
number_of_batches = samples/batch_size
X_t = Xf
# plt.imshow(X[1][10].reshape(14,14), cmap='gray')
for i in range(epochs):
    wm = np.zeros(Wm.shape)
    bm = np.zeros(Bm.shape)
    wc = np.zeros(Wc.shape)
    bc = np.zeros(Bc.shape)
    wd = np.zeros(Wd.shape)
    bd = np.zeros(Bd.shape)
    for j in range(number_of_batches):
        X = (X_t[j*batch_size:(j+1)*batch_size])
#         print(X.shape)
        loss = 0
        #forward pass
        outm = layer(X,Wm,Bm)
        mean  = relu(outm)
        outc = layer(X,Wc,Bc)
        cova  = relu(outc)
        gaus = np.random.normal(0,1,(cova.shape))
        Z    = mean + np.multiply((np.exp(0.5*cova)),gaus)
        outy = layer(Z,Wd,Bd)
        y    = sigm(outy)
        # Backprop decoder part 
        temp = diff_SSE(y,X,batch_size)*diff_sigm(outy)
        bd   += np.sum(temp,axis=0)
        wd   += np.matmul(Z.T,temp)
    
        ### Backprop Encoder part
        sct = -0.5*gaus*np.exp(0.5*cova)*np.matmul(temp,Wd.T)*diff_relu(outc)
        smt = -np.matmul(temp,Wd.T)*diff_relu(outm)
        sm = smt + 2*(mean)*diff_relu(outm)*Lambda
    #     print(smt.shape, outm.shape, cova.shape)
        sc = sct + 0.5*(np.exp(cova)-1)*diff_relu(outc)*Lambda
    #     print(smt)    
        ### 
    
        bm += np.sum(sm, axis=0).reshape(bm.shape)
        wm += np.matmul(X.T,sm)
    
        bc += np.sum(sc, axis=0).reshape(bc.shape)
        wc += np.matmul(X.T,sc)
    
        loss_1 = np.sum(np.linalg.norm(y-X, axis=0))
        loss_2 = np.sum(np.exp(cova)-1-cova + mean*mean ,axis=1)/batch_size
        loss   = loss_1 + np.sum(loss_2)
    
    Wm -= lr*wm
    Bm -= lr*bm
    Wc -= lr*wc
    Bc -= lr*bc
    Bd -= lr*bd
    Wd -= lr*wd

    print(loss, "epoch = ", i, "batch =", j)

(899.8496186693774, 'epoch = ', 0, 'batch =', 599)
(850.4838997129596, 'epoch = ', 1, 'batch =', 599)
(805.7807807381796, 'epoch = ', 2, 'batch =', 599)
(765.9449457314721, 'epoch = ', 3, 'batch =', 599)
(730.590339970294, 'epoch = ', 4, 'batch =', 599)
(699.5653430784918, 'epoch = ', 5, 'batch =', 599)
(672.1898831655749, 'epoch = ', 6, 'batch =', 599)
(648.1679684924679, 'epoch = ', 7, 'batch =', 599)
(626.9140563407058, 'epoch = ', 8, 'batch =', 599)
(608.0809000729639, 'epoch = ', 9, 'batch =', 599)
(591.3606625193395, 'epoch = ', 10, 'batch =', 599)
(576.4111196847148, 'epoch = ', 11, 'batch =', 599)
(562.9746597974278, 'epoch = ', 12, 'batch =', 599)
(550.8906872898183, 'epoch = ', 13, 'batch =', 599)
(539.9526289679427, 'epoch = ', 14, 'batch =', 599)
(529.9980151263486, 'epoch = ', 15, 'batch =', 599)
(520.9424525105427, 'epoch = ', 16, 'batch =', 599)
(512.6399258126312, 'epoch = ', 17, 'batch =', 599)
(505.02545166973937, 'epoch = ', 18, 'batch =', 599)
(498.01664735013856, '

(350.5412542108656, 'epoch = ', 156, 'batch =', 599)
(350.35858612253924, 'epoch = ', 157, 'batch =', 599)
(350.17885011435203, 'epoch = ', 158, 'batch =', 599)
(349.97561837397524, 'epoch = ', 159, 'batch =', 599)
(349.80891248274327, 'epoch = ', 160, 'batch =', 599)
(349.614164280713, 'epoch = ', 161, 'batch =', 599)
(349.4314404953425, 'epoch = ', 162, 'batch =', 599)
(349.24164729339196, 'epoch = ', 163, 'batch =', 599)
(349.068979454923, 'epoch = ', 164, 'batch =', 599)
(348.9009429781352, 'epoch = ', 165, 'batch =', 599)
(348.7073332894857, 'epoch = ', 166, 'batch =', 599)
(348.52352117241276, 'epoch = ', 167, 'batch =', 599)
(348.37611692842677, 'epoch = ', 168, 'batch =', 599)
(348.18830238366985, 'epoch = ', 169, 'batch =', 599)
(348.02005746099985, 'epoch = ', 170, 'batch =', 599)
(347.8747887623789, 'epoch = ', 171, 'batch =', 599)
(347.7096458472588, 'epoch = ', 172, 'batch =', 599)
(347.5318330717822, 'epoch = ', 173, 'batch =', 599)
(347.3685975755501, 'epoch = ', 174, 'b

In [ ]:
w=14
h=14
fig=plt.figure(figsize=(w/2,h/2))
columns = 4
rows = 4
for i in range(1, columns*rows +1):
    Z = np.random.normal(-10,10,(Bc.shape))
#     Z    = mean + np.multiply((np.exp(0.5*cova)),gaus)
    outy = layer(Z,Wd,Bd)
    y    = sigm(outy)
    img = y.reshape(w,h)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
'''
Wd = np.random.normal(0,1e-3,(100,input_size))
Bd = np.random.normal(0,1e-3,(1,input_size))

Wdi = np.random.normal(0,1,(mean,100))
Bdi = np.random.normal(0,1,(1,100))

#Here the covariance is log coavriance but named as coavriance for peace of mind while coding
lr = 1e-4
epochs = 300
batch_size = 60
number_of_batches = samples/batch_size
X_t = Xf.reshape(number_of_batches,batch_size,input_size)
# plt.imshow(X[1][10].reshape(14,14), cmap='gray')
for i in range(epochs):
    for j in range(number_of_batches):
        X = (X_t[j])
        # print(X.shape)
        loss = 0
        wm = np.zeros(Wm.shape)
        bm = np.zeros(Bm.shape)
        wc = np.zeros(Wc.shape)
        bc = np.zeros(Bc.shape)
        wd = np.zeros(Wd.shape)
        bd = np.zeros(Bd.shape)
        wdi = np.zeros(Wdi.shape)
        bdi = np.zeros(Bdi.shape)
        # forward pass
#         outi = layer(X,Wi,Bi)
#         aouti = sigm(outi)
        outm = layer(X,Wm,Bm)
        mean  = relu(outm)
        outc = layer(X,Wc,Bc)
        cova  = relu(outc)
    
    
        gaus = np.random.normal(0,1,(cova.shape))
        Z    = mean + np.multiply((np.exp(0.5*cova)),gaus)
        outyi = layer(Z,Wdi,Bdi)
        yi   = relu(outyi)
        outy = layer(yi,Wd,Bd)
        y = sigm(outy)
        
        # Backprop decoder part 
        temp = diff_SSE(y,X,batch_size)*diff_sigm(outy)
        bd   = np.sum(temp,axis=0)
        wd   = np.matmul(yi.T,temp)
        
        temp1 = backprop_layer_relu(temp,outyi,Wd)
        bdi  = np.sum(temp1, axis=0)
        wdi  = np.matmul(Z.T,temp1)
    
        ### Backprop Encoder part
        sct = -0.5*gaus*np.exp(0.5*cova)*backprop_layer_relu(temp1,np.exp(0.5*cova),Wdi)
        smt = -backprop_layer_relu(temp1,outm,Wdi)
        sm = smt + 2*(mean)*diff_relu(outm)                                                                            
    #     print(smt.shape, outm.shape, cova.shape)
        sc = sct + 0.5*(np.exp(cova)-1)*diff_relu(outc)
    #     print(smt)    
        ### 
    
        bm = np.sum(sm, axis=0).reshape(bm.shape)
        wm = np.matmul(X.T,sm)
    
        bc = np.sum(sc, axis=0).reshape(bc.shape)
        wc = np.matmul(X.T,sc)
        
        # we add up the sigmas coming from the previous layer to justify the divergence of 
        # one layer output to two others
        
        # Technically the functional output of the mean and variance layer to Z latent variable
        # Can be modelled as the identity and exp() functions and backprop can be done but gets a bit tedious
        # to define cases for all possible activations before that layer.
        
#         si = backprop_layer_sigm(sm,outi,Wm)+backprop_layer_sigm(sc,outi,Wc)
#         bi = np.sum(si, axis=0).reshape(bi.shape)
#         wi = np.matmul(X.T,si)
    
        loss_1 = np.sum(np.linalg.norm(y-X, axis=0))
        loss_2 = np.sum(np.exp(cova)-1-cova + mean*mean ,axis=1)
        loss   = loss_1 + np.sum(loss_2)
        
#         print(Wc.shape, wc.shape)
        
        Wm -= lr*wm
        Bm -= lr*bm
        Wc -= lr*wc
        Bc -= lr*bc
        Bd -= lr*bd
        Wd -= lr*wd
        Wdi -= lr*wdi
        Bdi -= lr*bdi
    print(loss)
'''